In [ ]:
!pip install --user tfx==0.21.1 tensorflow-data-validation==0.21.5

In [ ]:
from grpc import insecure_channel
from ml_metadata.proto import metadata_store_service_pb2
from ml_metadata.proto import metadata_store_service_pb2_grpc
import tensorflow_data_validation as tfdv

In [ ]:
channel = insecure_channel('metadata-grpc-service.kubeflow:8080')
stub = metadata_store_service_pb2_grpc.MetadataStoreServiceStub(channel)

In [ ]:
request = metadata_store_service_pb2.GetArtifactsRequest()
response = stub.GetArtifacts(request)

In [ ]:
last_id = 0
uri = ""
for artifact in response.artifacts:
    if artifact.custom_properties["pipeline_name"].string_value == "iris" and artifact.custom_properties["producer_component"].string_value == "StatisticsGen":
        if artifact.id > last_id:
            last_id = artifact.id
            uri = artifact.uri
uri

In [ ]:
train_stats = tfdv.load_statistics(uri + "/train/stats_tfrecord")
eval_stats = tfdv.load_statistics(uri + "/eval/stats_tfrecord")

tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')